In [ ]:
import numpy as np
from scipy.stats import norm, lognorm

def nonparametric_estimator(data, tau):
    n = len(data)
    k = int(np.ceil(tau * n))
    if k >= n:
        k = n - 1
    return np.partition(data, k)[k]

def parametric_estimator(data, tau, distribution):
    if distribution == 'normal':
        params = norm.fit(data)
        return norm.ppf(tau, *params)
    elif distribution == 'lognormal':
        params = lognorm.fit(data)
        return lognorm.ppf(tau, *params)
    else:
        raise ValueError('Invalid distribution specified.')

In [ ]:
demand_distributions = ['normal', 'lognormal']
sample_sizes = [10, 50, 100, 200]
target_service_levels = [0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 0.99]
param_values = [(110, 10), (100, 20)]
repetitions = 10

In [3]:
results = []

for distribution in demand_distributions:
    if distribution == 'normal':
        true_params = (120, 15)
    elif distribution == 'lognormal':
        true_params = (6, 0.6)
    
    for n in sample_sizes:
        for tau in target_service_levels:
            for param in param_values:
                rmse_npn = 0
                rmse_param = 0
                sl_npn = 0
                sl_param = 0
                plr_npn = 0
                plr_param = 0
                
                for _ in range(repetitions):
                    # Generate demand data
                    if distribution == 'normal':
                        demand_data = np.random.normal(*true_params, size=n)
                    elif distribution == 'lognormal':
                        demand_data = np.random.lognormal(*true_params, size=n)
                    
                    # Nonparametric estimator
                    order_quantity_npn = nonparametric_estimator(demand_data, tau)
                    rmse_npn += (order_quantity_npn - true_params[0])**2
                    
                    # Parametric estimator
                    order_quantity_param = parametric_estimator(demand_data, tau, distribution)
                    rmse_param += (order_quantity_param - true_params[0])**2
                    
                    sl_npn += int(order_quantity_npn >= demand_data.max())
                    sl_param += int(order_quantity_param >= demand_data.max())
                    
                    plr_npn += true_params[0] - np.mean(demand_data[demand_data <= order_quantity_npn])
                    plr_param += true_params[0] - np.mean(demand_data[demand_data <= order_quantity_param])
                
                rmse_npn /= repetitions
                rmse_param /= repetitions
                sl_npn /= repetitions
                sl_param /= repetitions
                plr_npn /= repetitions
                plr_param /= repetitions
                
                result = {
                    'Distribution': distribution,
                    'Sample Size': n,
                    'Target Service Level': tau,
                    'Param Values': param,
                    'RMSE NPN': np.sqrt(rmse_npn),
                    'RMSE Param': np.sqrt(rmse_param),
                    'SL NPN': sl_npn,
                    'SL Param': sl_param,
                    'PLR NPN': plr_npn,
                    'PLR Param': plr_param
                }
                print(result)
                
                results.append(result)

{'Distribution': 'lognormal', 'Sample Size': 10, 'Target Service Level': 0.99, 'Param Values': (100, 20), 'RMSE NPN': 883.7139610687067, 'RMSE Param': 412502174242.866, 'SL NPN': 1.0, 'SL Param': 1.0, 'PLR NPN': -429.8995005051147, 'PLR Param': -429.8995005051147}
{'Distribution': 'lognormal', 'Sample Size': 50, 'Target Service Level': 0.01, 'Param Values': (110, 10), 'RMSE NPN': 142.36059234539374, 'RMSE Param': 100.12081048046974, 'SL NPN': 0.0, 'SL Param': 0.0, 'PLR NPN': -119.09177493608213, 'PLR Param': nan}
{'Distribution': 'lognormal', 'Sample Size': 50, 'Target Service Level': 0.01, 'Param Values': (100, 20), 'RMSE NPN': 138.53885756017817, 'RMSE Param': 112.05838094726396, 'SL NPN': 0.0, 'SL Param': 0.0, 'PLR NPN': -123.96600168616851, 'PLR Param': nan}
{'Distribution': 'lognormal', 'Sample Size': 50, 'Target Service Level': 0.05, 'Param Values': (110, 10), 'RMSE NPN': 171.0325232193298, 'RMSE Param': 130.17012252308373, 'SL NPN': 0.0, 'SL Param': 0.0, 'PLR NPN': -137.20655840